In [1]:
import sys
import os
parent_dir = os.path.abspath('..')
sys.path.append(parent_dir)
import kinodyn_um.urdfparser as u2c
import numpy as np
import casadi as ca

import matplotlib.pyplot as plt

In [2]:
# ─────────────────────────────────────────────────────────────────────────────
# 1. Load robot model
# ─────────────────────────────────────────────────────────────────────────────
alpha = u2c.URDFparser()
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
path_to_urdf = os.path.join(project_root, 'usage', 'urdf', 'alpha_5_robot.urdf')
alpha.from_file(path_to_urdf)

root = "base_link"
tip  = "alpha_standard_jaws_base_link"
n_joints = alpha.get_n_joints(root, tip)

F_next = alpha.forward_simulation(root, tip)

F_next.save('arm.casadi')

# c , cpp or matlab code generation for forward dynamics
F_next.generate("F_next_.c")
os.system(f"gcc -fPIC -shared F_next_.c -o libMnext.so")

print('✔ arm.casadi saved — ready for simulation')

✔ arm.casadi saved — ready for simulation


In [3]:
# fixed sizes -------------------------------------------------------------
n_theta   = 32
nx        = 2*n_joints                      # 8   (q ‖ q̇)
nv        = n_joints                        # 4   velocity rows
lam       = 1e-4                            # Tikhonov damping
N         = 50                              # prediction horizon (set as you like)

# ------------------------------------------------------------------------
# 1)  STEP function: one forward‑dynamics step
# ------------------------------------------------------------------------
x_prev   = ca.MX.sym("x_prev",  nx)
tau_k    = ca.MX.sym("tau_k",   n_joints)

# constant‑within‑horizon symbols
dt        = ca.MX.sym("dt")
g         = ca.MX.sym("g")
payload   = ca.MX.sym("payload_props", 4)
theta     = ca.MX.sym("theta",  n_theta)
low_lim   = ca.MX.sym("low_lim", n_joints)
up_lim    = ca.MX.sym("up_lim", n_joints)
eps_T     = ca.MX.sym("eps_T",  n_joints)

x_next = F_next(
    x_prev, tau_k,
    dt, g, payload,
    theta, low_lim, up_lim, eps_T
)

step = ca.Function(
    "step",
    [x_prev, tau_k, dt, g, payload, theta, low_lim, up_lim, eps_T],
    [x_next]
)

# ------------------------------------------------------------------------
# 2)  ROLLOUT with mapaccum  (serial scan)
# ------------------------------------------------------------------------
rollout = step.mapaccum(N)   # returns stack of N states

#  inputs to the rollout --------------------------------------------------
x0          = ca.MX.sym("x0", nx)               # single initial state
tau_seq     = ca.MX.sym("tau_seq", n_joints, N) # τ₀ … τ_{N‑1}
dt_seq      = ca.MX.sym("dt", 1, N)
g_sym       = ca.MX.sym("g")
payload_sym = ca.MX.sym("payload_props", 4)
theta_prev  = ca.MX.sym("theta_prev", n_theta)
low_lim_sym = ca.MX.sym("low_lim",  n_joints)
up_lim_sym  = ca.MX.sym("up_lim",   n_joints)
epsT_sym    = ca.MX.sym("eps_T",    n_joints)
theta_true  = ca.MX.sym("theta_true",  n_theta, 1)
P_prev     = ca.MX.sym("P_prev",     n_theta, n_theta)
z          = ca.MX.sym("z",          n_theta, 1)

X_pred = rollout(
    x0,          # initial state
    tau_seq,     # varying input  (n_joints × N)
    dt_seq, g_sym, payload_sym,
    theta_prev, low_lim_sym, up_lim_sym, epsT_sym
)                       # shape:  nx × N   (states x₁ … x_N)

# ------------------------------------------------------------------------
# 3)  build residual stack (velocity only) and Jacobian wrt θ
# ------------------------------------------------------------------------
v_meas = ca.MX.sym("v_meas", 4, N)    # measured x₁ … x_N
v_pred  = X_pred[4:8, :]                         # rows 4‑7  -> nv × N

# ‑‑ plain residual -------------------------------------------------------
err = v_meas - v_pred                     # shape: (nv, N)

# # ‑‑ per‑row RMS magnitude of the prediction (acts as scale) -------------
epsilon   = 1e-8
# rms_scale = ca.sqrt( ca.sum2(v_pred**2) / N + epsilon )   # (nv,1)

# # ‑‑ normalise **before** squaring ---------------------------------------
# res_norm       = err / ca.repmat(rms_scale, 1, N)         # dimensionless
err_metric     = ca.sum2(err**2)                     # scalar cost


# 2) excitation metric
u_norm2 = ca.sum1(ca.sum2(tau_seq**2))  # sum2 → (n_u×1), then sum1 → (1×1)
k       = 2.0

# --- adaptive gain -------------------------------------------------------
η       = 0.08                                    # tune 0.01‑0.2
alpha   = u_norm2 / (u_norm2 + k + epsilon)
alpha   = ca.fmin(ca.fmax(alpha, 0), 1)

gain    = η * alpha                               # effective step size

# --- mean & covariance update -------------------------------------------
mu_next = theta_prev + gain * (theta_true - theta_prev)
P_next  = (1 - gain) * P_prev

update_theta = ca.Function(
    "update_theta",
    [theta_prev, P_prev, x0, v_meas, tau_seq,
     dt_seq, g_sym, payload_sym, low_lim_sym, up_lim_sym, epsT_sym, theta_true, z],
    [mu_next, P_next, err_metric],
)


# ------------------------------------------------------------------------
# 6)  codegen
# ------------------------------------------------------------------------
update_theta.generate("manipulator_update_theta.c")
os.system("gcc -fPIC -shared manipulator_update_theta.c -o lib_manipulator_update_theta.so")
print("✔  lib_manipulator_update_theta.so built")


✔  lib_manipulator_update_theta.so built
